# Project - Airline AI Assistant

We'll now bring together what we've learned to make an AI Customer Support assistant for an Airline

In [1]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import sqlite3

In [2]:
# Initialization

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

# As an alternative, if you'd like to use Ollama instead of OpenAI
# Check that Ollama is running for you locally (see week1/day2 exercise) then uncomment these next 2 lines
# MODEL = "llama3.2"
# openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')


OpenAI API Key exists and begins sk-proj-


In [ ]:
system_message = "You are a helpful assistant for an Airline and Hotel Managenment Company called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [ ]:
# This function looks rather simpler than the one from my video, because we're taking advantage of the latest Gradio updates

def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content

gr.ChatInterface(fn=chat, type="messages").launch()

## Tools

Tools are an incredibly powerful feature provided by the frontier LLMs.

With tools, you can write a function, and have the LLM call that function as part of its response.

Sounds almost spooky.. we're giving it the power to run code on our machine?

Well, kinda.

In [27]:
# Let's start by making a useful function

ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [60]:
def get_hotel_price(destination_city):
    print(f"Tool get_hotel_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [43]:
get_ticket_price("Berlin")

Tool get_ticket_price called for Berlin


'$499'

In [44]:
# There's a particular dictionary structure that's required to describe our function:

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [45]:
# There's a particular dictionary structure that's required to describe our function:

hotel_price_function = {
    "name": "get_hotel_price",
    "description": "Get the price of a hotel charges to the destination city. Call this whenever you need to know the hotel price, for example when a customer asks 'How much does hotel charges in this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [46]:
# And this is included in a list of tools:

tools = [{"type": "function", "function": price_function}, {"type": "function", "function": hotel_price_function}]

[{'role': 'system', 'content': "You are a helpful assistant for an Airline and Hotel Managenment Company called FlightAI. Give short, courteous answers, no more than 1 sentence. Always be accurate. If you don't know the answer, say so."}, {'role': 'user', 'metadata': None, 'content': 'Hello I am planning to visit berlin, what will be hotel prices?', 'options': None}, {'role': 'user', 'content': 'Retry'}]
ChatCompletion(id='chatcmpl-BmvBqHUdK3QyRRPUuzX83daFA250R', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_Ai5ozH8pFlB0EmAga14N37Ae', function=Function(arguments='{"destination_city":"Berlin"}', name='get_hotel_price'), type='function')]))], created=1750999282, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_34a54ae93c', usage=Completion

## Getting OpenAI to use our Tool

There's some fiddly stuff to allow OpenAI "to call our tool"

What we actually do is give the LLM the opportunity to inform us that it wants us to run the tool.

Here's how the new chat function looks:

In [67]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    print(messages)
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    print(response)
    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        multi_response, city = handle_tool_call(message)
        messages.append(message)
        for response in multi_response:
            messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

In [68]:
# We have to write that function handle_tool_call:

def handle_tool_call(message):
    print(message)
    # tool_call = message.tool_calls[0]
    response = []
    for tool_call in message.tool_calls:
        arguments = json.loads(tool_call.function.arguments)
        # print(arguments)
        city = arguments.get('destination_city')
        function_tool_call = tool_call.function.name
        print(function_tool_call, city)
        if function_tool_call == 'get_ticket_price':
            price = get_ticket_price(city)
        else:
            price = get_hotel_price(city)
        response.append({
            "role": "tool",
            "content": json.dumps({"destination_city": city,"price": price}),
            "tool_call_id": tool_call.id
        })
    return response, city

In [69]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7882
* To create a public link, set `share=True` in `launch()`.


[{'role': 'system', 'content': "You are a helpful assistant for an Airline and Hotel Managenment Company called FlightAI. Give short, courteous answers, no more than 1 sentence. Always be accurate. If you don't know the answer, say so."}, {'role': 'user', 'content': 'plane and hotel price london'}]
ChatCompletion(id='chatcmpl-BmvMdcp6yaRZdMfBNAFu87RR4Vdcd', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_tMMtRp78Ml8R6PQoL4KPRfqs', function=Function(arguments='{"destination_city": "London"}', name='get_ticket_price'), type='function'), ChatCompletionMessageToolCall(id='call_jL8A4k5xE656p57Qy7s2lSmU', function=Function(arguments='{"destination_city": "London"}', name='get_hotel_price'), type='function')]))], created=1750999951, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tie

In [142]:
con = sqlite3.connect('flight_data.db')
cur = con.cursor()
cur.execute('''Drop Table flight_data;''')
con.commit()
con.close()

In [123]:
con = sqlite3.connect('flight_data.db')
cur = con.cursor()
cur.execute('''CREATE TABLE flight_data (
    id SERIAL PRIMARY KEY,                      -- Auto-increment ID for each record
    airline VARCHAR(100) NOT NULL,              -- Name of the airline
    flight VARCHAR(50) NOT NULL,                -- Flight number/code
    source_city VARCHAR(100) NOT NULL,          -- City of departure
    departure_time TIME NOT NULL,               -- Scheduled departure time
    stops VARCHAR(50),                          -- Number/type of stops (e.g., 'non-stop', '1 stop')
    arrival_time TIME NOT NULL,                 -- Scheduled arrival time
    destination_city VARCHAR(100) NOT NULL,     -- City of arrival
    class VARCHAR(50) NOT NULL,                 -- Travel class (e.g., Economy, Business)
    duration INTERVAL,                          -- Duration of the flight (e.g., '2 hours 30 minutes')
    days_left INT CHECK (days_left >= 0),       -- Days left until departure
    price DECIMAL(10, 2) NOT NULL               -- Ticket price
);''')
con.commit()
con.close()


In [128]:
con = sqlite3.connect('flight_data.db')
cur = con.cursor()
for row in cur.execute("select * from flight_data"):
    print(row)

(None, 'SpiceJet', 'SG-8709', 'Delhi', 'Evening', 'zero', 'Night', 'Mumbai', 'Economy', 2.17, 1, 5953)
(None, 'SpiceJet', 'SG-8157', 'Delhi', 'Early_Morning', 'zero', 'Morning', 'Mumbai', 'Economy', 2.33, 1, 5953)
(None, 'AirAsia', 'I5-764', 'Delhi', 'Early_Morning', 'zero', 'Early_Morning', 'Mumbai', 'Economy', 2.17, 1, 5956)
(None, 'Vistara', 'UK-995', 'Delhi', 'Morning', 'zero', 'Afternoon', 'Mumbai', 'Economy', 2.25, 1, 5955)
(None, 'Vistara', 'UK-963', 'Delhi', 'Morning', 'zero', 'Morning', 'Mumbai', 'Economy', 2.33, 1, 5955)
(None, 'Vistara', 'UK-945', 'Delhi', 'Morning', 'zero', 'Afternoon', 'Mumbai', 'Economy', 2.33, 1, 5955)
(None, 'Vistara', 'UK-927', 'Delhi', 'Morning', 'zero', 'Morning', 'Mumbai', 'Economy', 2.08, 1, 6060)
(None, 'Vistara', 'UK-951', 'Delhi', 'Afternoon', 'zero', 'Evening', 'Mumbai', 'Economy', 2.17, 1, 6060)
(None, 'GO_FIRST', 'G8-334', 'Delhi', 'Early_Morning', 'zero', 'Morning', 'Mumbai', 'Economy', 2.17, 1, 5954)
(None, 'GO_FIRST', 'G8-336', 'Delhi', 'A

In [92]:
import pandas as pd
df = pd.read_csv('./Clean_Dataset.csv')
df = df[['airline', 'flight', 'source_city', 'departure_time',
       'stops', 'arrival_time', 'destination_city', 'class', 'duration',
       'days_left', 'price']]

In [125]:
df.shape
df.columns

Index(['airline', 'flight', 'source_city', 'departure_time', 'stops',
       'arrival_time', 'destination_city', 'class', 'duration', 'days_left',
       'price'],
      dtype='object')

In [144]:
import pandas as pd
from sqlalchemy import create_engine
con = sqlite3.connect('flight_data.db')
cur = con.cursor()
# SQLite (local file)
engine = create_engine('sqlite:///flight_data.db')  # Creates 'flights.db' in your directory

# Push data to 'flight_data' table
df.to_sql('flight_data', engine, if_exists='append', index=False)

print("Data successfully inserted into flight_data table.")
con.commit()
con.close()


Data successfully inserted into flight_data table.


In [3]:
def fetch_airplane_data(query):
    con = sqlite3.connect('flight_data.db')
    cur = con.cursor()
    result = cur.execute(query).fetchall()
    # # print(cur.fetchall())
    # for x in cur.fetchall():
    #     print(x)
    cur.close()
    return result

In [64]:
def success(bool_flag):
    print('Called')
    if bool_flag == 'True':
        return True
    else:
        return False
    

In [65]:
airplane_data_function = {
    "name": "fetch_airplane_data",
    "description": ''' 
                Based on a sql query, the function returns the data from table flight_data.
    Columns details of flight_data Table
                1) Airline: The name of the airline company is stored in the airline column. It is a categorical feature having 6 different airlines.
            2) Flight: Flight stores information regarding the plane's flight code. It is a categorical feature.
            3) Source City: City from which the flight takes off. It is a categorical feature having 6 unique cities.
            4) Departure Time: This is a derived categorical feature obtained created by grouping time periods into bins. It stores information about the departure time and have 6 unique time labels.
            5) Stops: A categorical feature with 3 distinct values that stores the number of stops between the source and destination cities.
            6) Arrival Time: This is a derived categorical feature created by grouping time intervals into bins. It has six distinct time labels and keeps information about the arrival time.
            7) Destination City: City where the flight will land. It is a categorical feature having 6 unique cities.
            8) Class: A categorical feature that contains information on seat class; it has two distinct values: Business and Economy.
            9) Duration: A continuous feature that displays the overall amount of time it takes to travel between cities in hours.
            10)Days Left: This is a derived characteristic that is calculated by subtracting the trip date by the booking date.
            11) Price: Target variable stores information of the ticket price.
    ''',
    "parameters": {
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "query to for sql table to extract insights",
            },
        },
        "required": ["query"],
        "additionalProperties": False
    }
}

In [66]:
success_function = {
    "name": "success",
    "description": ''' if you are satisfied with data analysis return text 'True' else 'False' to run different set of query
    ''',
    "parameters": {
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "boolean to continue or stop querying the db",
            },
        },
        "required": ["bool_flag"],
        "additionalProperties": False
    }
}

In [67]:
def handle_tool_call(message):
    # print(message)
    # tool_call = message.tool_calls[0]
    response = []
    # stop_flag = False
    for tool_call in message.tool_calls:
        arguments = json.loads(tool_call.function.arguments)
        # print(arguments)
        
        query = arguments.get('query')
        output = fetch_airplane_data(query)
            
        response.append({
            "role": "tool",
            "content": json.dumps({"query": query,"output": output}),
            "tool_call_id": tool_call.id
        })
    print(response)
    return response, query

In [72]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    # print(messages)
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    print(response)
    # print(tools)
    # print(response)
    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        print('Hi')
        multi_response, query, stop_flag = handle_tool_call(message)
        messages.append(message)
        for response in multi_response:
            messages.append(response)
        print(messages)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
        print('------/n')
        print(response)
        print('------')
    return response.choices[0].message.content

In [73]:
system_message = "You are a Data Analyst with good SQL command working for Airline Company FirebaseAI. "
system_message += "Your role is to write sql query which can be used to extract data from fight_data db which has following information: 'airline', 'flight', 'source_city', \
                    'departure_time', 'stops', 'arrival_time', 'destination_city', 'class', 'duration', 'days_left','price' .\
                    You can iteratively run database database to answer questions which may require multiple runs. You must answer only using the DB"
system_message += "If you don't know the answer, say so."

In [74]:
tools = [{'type':'function', 'function':airplane_data_function}]

In [75]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7880
* To create a public link, set `share=True` in `launch()`.


ChatCompletion(id='chatcmpl-BnKtjZuN0bxKOo2h2Y8fuRCwcy8hp', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_JFOzfXlSi38ZWuQ5oHOJBXSc', function=Function(arguments='{"query": "SELECT AVG(price) AS avg_price FROM flight_data WHERE source_city = \'Chennai\' AND destination_city = \'Mumbai\'"}', name='fetch_airplane_data'), type='function'), ChatCompletionMessageToolCall(id='call_AWyYPVqqAMOweUDQR8GyLuSu', function=Function(arguments='{"query": "SELECT AVG(price) AS avg_price FROM flight_data WHERE source_city = \'Mumbai\' AND destination_city = \'Delhi\'"}', name='fetch_airplane_data'), type='function'), ChatCompletionMessageToolCall(id='call_xtk89d67G02DB8fro3W4mL11', function=Function(arguments='{"query": "SELECT AVG(price) AS avg_price FROM flight_data WHERE source_city = \'Delhi\' AND destinat